In [15]:
import numpy as np
import tensorflow as tf
import kerastuner as kt
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline 
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers import Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score
from sklearn. model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import sklearn.metrics as metrics

In [2]:

df= pd.read_csv("all_data_LSTM_full_feature.csv")
test=pd.read_csv("y_DX_month.csv")
df=df.sort_values(by='RID', ascending=True)
test=test.sort_values(by='RID', ascending=True)
groupby=df.groupby("RID").count()
len(groupby)
df=df.drop(['RID', 'VISCODE2'],axis = 1 )


In [3]:
num_features=len(df.columns)
y= np.array(test.DX_Month)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(df)
X = scaler.transform(df)

In [4]:

scaler2 = MinMaxScaler(feature_range=(0, 1))
y = scaler2.fit_transform(pd.DataFrame(y))


In [5]:
x= X.reshape(613, 4, num_features)
X_train, X_test, y_train, y_test = train_test_split(
     x, y, test_size=0.10, stratify= y, shuffle=True, random_state=None) 

In [6]:

def build_model(hp):
    num_units_min  =  300
    num_units_max  =  700
    num_units_step =  20

    dropout_min  =  .2
    dropout_max  =  0.5
    dropout_step =  0.1
    
    model = keras.Sequential()
    model.add(layers.LSTM(units=hp.Int('unit1',  min_value=num_units_min,
                                                 max_value=num_units_max,
                                                 step=num_units_step),
                                                 activation='relu',
                                                 input_shape=(X_train.shape[1], X_train.shape[2]), 
                                                 kernel_initializer='normal', 
                                                 kernel_regularizer=keras.regularizers.l2(hp.Choice('reg_rate',values=[0.01, 0.05, 0.1])),
                          
                                                 return_sequences = True))
    model.add(layers.Dropout(hp.Float('dropout_1',min_value=dropout_min,
                                      max_value=dropout_max,
                                      step=dropout_step) ) )
 
  
    model.add(layers.Flatten())
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(layers.Dense(units=hp.Int('unitsdense_'+ str(i),
                                            min_value=50,
                                            max_value=100,
                                            step=10),
                                            activation='relu',
                              kernel_regularizer=keras.regularizers.l2(hp.Choice('reg_rate1'+ str(i),values=[0.01, 0.05, 0.1]))))   
        model.add(layers.Dropout(hp.Float('dropoutdense_'+ str(i), 
                                      min_value=dropout_min,
                                      max_value=dropout_max,
                                      step=dropout_step)))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[ 1e-4])),
        loss='mean_squared_error',
        metrics=['mean_squared_error', 'mean_absolute_error'])
    return model


In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping

tuner = RandomSearch(
    build_model,
    objective='mean_squared_error',
    max_trials=4,
  
    project_name='hager4')



INFO:tensorflow:Reloading Oracle from existing project .\hager4\oracle.json
INFO:tensorflow:Reloading Tuner from .\hager4\tuner0.json


In [8]:
early_stopping = EarlyStopping(monitor='mean_squared_error', patience=30, verbose=1)
callback_list = [ early_stopping ]
tuner.search(X_train, y_train,
             epochs=60,
             batch_size=50, 
             callbacks=callback_list,
             validation_data=(X_test,y_test))
model = tuner.get_best_models(num_models=1)[0]


INFO:tensorflow:Oracle triggered exit

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x0000012512469820> and <tensorflow.python.keras.layers.core.Dropout object at 0x000001251823E6A0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000125181C1D60> and <tensorflow.python.keras.layers.core.Dropout object at 0x00000125181E5850>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000125181ECDF0> and <tensorflow.python.keras.layers.core.Dropout object at 0x00000125181ECAC0>).


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4, 440)            1008480   
_________________________________________________________________
dropout (Dropout)            (None, 4, 440)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1760)              0         
_________________________________________________________________
dense (Dense)                (None, 60)                105660    
_________________________________________________________________
dropout_1 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 80)                4880      
_________________________________________________________________
dropout_2 (Dropout)          (None, 80)                0

In [10]:
tuner.get_best_hyperparameters()[0].values

{'unit1': 440,
 'reg_rate': 0.1,
 'dropout_1': 0.2,
 'num_layers': 2,
 'unitsdense_0': 60,
 'reg_rate1': 0.01,
 'dropoutdense_0': 0.30000000000000004,
 'learning_rate': 0.0001,
 'unitsdense_1': 80,
 'dropoutdense_1': 0.30000000000000004,
 'unitsdense_2': 70,
 'dropoutdense_2': 0.2,
 'reg_rate10': 0.01,
 'reg_rate11': 0.01}

In [11]:
best_model = tuner.get_best_models()[0]



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x0000012515E44520> and <tensorflow.python.keras.layers.core.Dropout object at 0x0000012518206940>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000125183E46D0> and <tensorflow.python.keras.layers.core.Dropout object at 0x00000125183E86A0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x00000125183F3B80> and <tensorflow.python.keras.layers.core.Dropout object at 0x00000125183F3F40>).


In [12]:
my_dict_Train={"RMSE": [],"MSE": [],"MBE": []};
my_dict_Test={"RMSE": [],"MSE": [],"MBE": []};

In [16]:
for i in range(0,2):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.10, stratify= y, shuffle=True, random_state=None)
        history= best_model.fit(X_train, y_train, epochs=15, batch_size=50,  verbose=1,shuffle=True, validation_data=(X_test,y_test))
        my_dict_Train["RMSE"].append(np.sqrt(np.mean(history.history['val_mean_squared_error'])) )
        my_dict_Train["MSE"].append(np.mean(history.history['val_mean_squared_error']))
        my_dict_Train["MBE"].append(np.mean(history.history['val_mean_absolute_error']))
        y_pred = best_model.predict(X_test)
        my_dict_Test["RMSE"].append(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
        my_dict_Test["MSE"].append(metrics.mean_squared_error(y_test,y_pred))
        my_dict_Test["MBE"].append(metrics.mean_absolute_error(y_test,y_pred))
        best_model.reset_states()

Epoch 1/15
12/12 [==============================] - 1s 56ms/step - loss: 2.5150 - mean_squared_error: 0.0591 - mean_absolute_error: 0.1520 - val_loss: 2.4602 - val_mean_squared_error: 0.0561 - val_mean_absolute_error: 0.1580
Epoch 2/15
12/12 [==============================] - 1s 47ms/step - loss: 2.4273 - mean_squared_error: 0.0595 - mean_absolute_error: 0.1583 - val_loss: 2.3739 - val_mean_squared_error: 0.0560 - val_mean_absolute_error: 0.1553
Epoch 3/15
12/12 [==============================] - 1s 47ms/step - loss: 2.3414 - mean_squared_error: 0.0582 - mean_absolute_error: 0.1678 - val_loss: 2.2919 - val_mean_squared_error: 0.0566 - val_mean_absolute_error: 0.1634
Epoch 4/15
12/12 [==============================] - 1s 46ms/step - loss: 2.2603 - mean_squared_error: 0.0587 - mean_absolute_error: 0.1485 - val_loss: 2.2115 - val_mean_squared_error: 0.0563 - val_mean_absolute_error: 0.1453
Epoch 5/15
12/12 [==============================] - 1s 45ms/step - loss: 2.1825 - mean_squared_error

In [17]:
print("Training result")
for metric in my_dict_Train.keys():
    print("%s: %.3f" % (metric, np.mean(my_dict_Train[metric])))
   

Training result
RMSE: 0.243
MSE: 0.059
MBE: 0.152


In [18]:
print("Testing result")
for metric in my_dict_Test.keys():
    print("%s: %.3f" % (metric, np.mean(my_dict_Test[metric])))

Testing result
RMSE: 0.242
MSE: 0.059
MBE: 0.153
